# Makers

In this tutorial, you will:

- Learn about `Maker`s.
- Understand how to use a `Maker` to update the parameters of jobs in a flow.


## What is a `Maker`?

A `Maker` is class that makes it convenient to update parameters on-the-fly in a workflow. It is best explained by example.

Let's start by defining a simple `Maker` that either adds or multiplies two numbers together, which we will do twice to make a flow. Note that all classes inheriting from the `Maker` base class must have a `name` variable and a `make` method.


In [16]:
from dataclasses import dataclass
from jobflow import job, Flow, Maker
from jobflow.managers.local import run_locally

@dataclass
class AddMaker(Maker):
    name: str = "Add Maker"
    operation: str = "add"

    @job
    def make(self, a, b):
        if self.operation == "add":
            return a + b
        elif self.operation == "mult":
            return a * b
        else:
            raise ValueError(f"Unknown operation: {self.operation}")


job1 = AddMaker().make(a=2, b=3)
job2 = AddMaker().make(a=job1.output, b=4)

flow = Flow([job1, job2])
responses = run_locally(flow)

2023-06-07 23:10:04,593 INFO Started executing jobs locally
2023-06-07 23:10:04,596 INFO Starting job - Add Maker (fb2f2c62-2858-43e2-93cc-ca0eb7247278)
2023-06-07 23:10:04,597 INFO Finished job - Add Maker (fb2f2c62-2858-43e2-93cc-ca0eb7247278)
2023-06-07 23:10:04,598 INFO Starting job - Add Maker (b6ced349-79ec-44fb-bb3c-829817404353)
2023-06-07 23:10:04,600 INFO Finished job - Add Maker (b6ced349-79ec-44fb-bb3c-829817404353)
2023-06-07 23:10:04,601 INFO Finished executing jobs locally


In [17]:
for uuid, response in responses.items():
    print(f"{uuid} -> {response}")

fb2f2c62-2858-43e2-93cc-ca0eb7247278 -> {1: Response(output=5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
b6ced349-79ec-44fb-bb3c-829817404353 -> {1: Response(output=9, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


Right now, nothing particularly special has happened here. But what if you had a much more complex workflow with many steps and you wanted to change the `AddMaker` keyword arguments, but only for a few individual jobs in the flow? That's where the `Maker` comes in handy. Let's see how it works.


In [18]:
from dataclasses import dataclass
from jobflow import job, Flow, Maker
from jobflow.managers.local import run_locally

@dataclass
class AddMaker(Maker):
    name: str = "Add Maker"
    operation: str = "add"

    @job
    def make(self, a, b):
        if self.operation == "add":
            return a + b
        elif self.operation == "mult":
            return a * b
        else:
            raise ValueError(f"Unknown operation: {self.operation}")

@dataclass
class SubtractMaker(Maker):
    name: str = "Subtract Maker"

    @job
    def make(self, a, b):
        return b - a

job1 = AddMaker().make(a=2, b=3)
job2 = SubtractMaker().make(a=job1.output, b=4)
flow = Flow([job1, job2])

In [19]:
flow.update_maker_kwargs({"operation": "mult"}, name_filter="Add Maker")
responses = run_locally(flow)

2023-06-07 23:10:04,843 INFO Started executing jobs locally
2023-06-07 23:10:04,844 INFO Starting job - Add Maker (a4442883-8280-40eb-a153-d7d22ad29a54)
2023-06-07 23:10:04,845 INFO Finished job - Add Maker (a4442883-8280-40eb-a153-d7d22ad29a54)
2023-06-07 23:10:04,847 INFO Starting job - Subtract Maker (c061ea2f-698e-4ab6-adc6-ae014b7ef3eb)
2023-06-07 23:10:04,848 INFO Finished job - Subtract Maker (c061ea2f-698e-4ab6-adc6-ae014b7ef3eb)
2023-06-07 23:10:04,849 INFO Finished executing jobs locally


In [20]:
for uuid, response in responses.items():
    print(f"{uuid} -> {response}")

a4442883-8280-40eb-a153-d7d22ad29a54 -> {1: Response(output=6, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
c061ea2f-698e-4ab6-adc6-ae014b7ef3eb -> {1: Response(output=-2, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


In this example, we have updated the keyword arguments ("kwargs") of jobs in a flow using a `name_filter` and the `update_maker_kwargs` function, which functions because the classes in the flow are themselves `Maker` objects.

Of course, we could have simply done `job1 = AddMaker(operation="mult").make(a=2, b=3)` to begin with, but in practice if you were to have instead impotred this flow from some external Python package, you might not be able to modify the `AddMaker` class directly. In this case, the `Maker` class provides a convenient way to update the parameters of jobs in a flow without having to redefine the flow itself.